In [2]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster('local').setAppName('spark_sql_basic')
sc = SparkContext(conf = conf)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/07 02:08:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
movies_rdd = sc.parallelize([
    (1, ("어벤져스", "마블")),
    (2, ("슈퍼맨", "DC")),
    (3, ("배트맨", "DC")),
    (4, ("겨울왕국", "디즈니")),
    (5, ("아이언맨", "마블"))
])


attendances_rdd = sc.parallelize([
    (1, (13934592, "KR")),
    (2, (2182227,"KR")),
    (3, (4226242, "KR")),
    (4, (10303058, "KR")),
    (5, (4300365, "KR"))
])

마블 영화 중 관객 수가 500만 이상인 영화 가져오기
1. Inner Join -> Filter By Movies -> Filter By Attendance
2. Filter By Movies, Attendance -> Inner Join

In [4]:
# Case 1. Join 먼저, filter By 나중에
movie_att = movies_rdd.join(attendances_rdd)
movie_att.collect()

[(2, (('슈퍼맨', 'DC'), (2182227, 'KR'))),
 (4, (('겨울왕국', '디즈니'), (10303058, 'KR'))),
 (1, (('어벤져스', '마블'), (13934592, 'KR'))),
 (3, (('배트맨', 'DC'), (4226242, 'KR'))),
 (5, (('아이언맨', '마블'), (4300365, 'KR')))]

In [5]:
movie_att.filter(
    lambda x : x[1][0][1] == '마블' and x[1][1][0] >= 5000000
).collect()

[(1, (('어벤져스', '마블'), (13934592, 'KR')))]

In [6]:
# Case2 . Filter 먼저, Join 나중에

In [7]:
filtered_movies = movies_rdd.filter(
    lambda x : x[1][1] == '마블'
)

In [8]:
filtered_att = attendances_rdd.filter(
        lambda x : x[1][0] >=5000000
)

In [9]:
filtered_movies.join(filtered_att).collect()

[(1, (('어벤져스', '마블'), (13934592, 'KR')))]

In [11]:
sc.stop()

NameError: name 'spark' is not defined

# spark SQL

In [13]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').appName('spark-sql').dgetOrCreate()
spark
# getOrCreate(): 객체를 만듦(기능들을 구현하기 위한 객체) 

RDD를 이용해서 데이터프레임 생성하기

In [14]:
movies = [
    (1, "어벤져스", "마블", 2012, 4, 26),
    (2, "슈퍼맨", "DC", 2013, 6, 13),
    (3, "배트맨", "DC", 2008, 8, 6),
    (4, "겨울왕국", "디즈니", 2014, 1, 16),
    (5, "아이언맨", "마블", 2008, 4, 30)
]

movie_schema = ["id", "name", "company", "year", "month", "day"]

## 데이터프레임 생성
- inferSchema 옵션을 활용하면 타입을 스파크가 자동으로 결정(기본)

In [15]:
df = spark.createDataFrame(data=movies, schema=movie_schema) 
# 스파크 데이터 프레임을 만듦

In [16]:
# 스키마 확인
df.dtypes

[('id', 'bigint'),
 ('name', 'string'),
 ('company', 'string'),
 ('year', 'bigint'),
 ('month', 'bigint'),
 ('day', 'bigint')]

In [17]:
# dataframe은 곧 rdd 
df

DataFrame[id: bigint, name: string, company: string, year: bigint, month: bigint, day: bigint]

전체 데이터프레임의 내용을 확인
- show()

In [18]:
df.show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  4|겨울왕국| 디즈니|2014|    1| 16|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



## 데이터프레임에 SQL 사용하기
- 데이터 프레임을 temporary view에 등록해야 Spark SQL 을 사용 가능

In [19]:
df.createOrReplaceTempView("movies") # movies라는 임시 테이블 생성
# sql을 활용하기 위한 테이블로 만들어줌 

movies 테이블에서 영화 이름(name)만 가져오기

In [20]:
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- company: string (nullable = true)
 |-- year: long (nullable = true)
 |-- month: long (nullable = true)
 |-- day: long (nullable = true)



In [21]:
query = """

SELECT name
FROM movies

"""

# 쿼리 실행
spark.sql(query).show()

+--------+
|    name|
+--------+
|어벤져스|
|  슈퍼맨|
|  배트맨|
|겨울왕국|
|아이언맨|
+--------+



In [22]:
# 2010년 이후에 개봉한 영화를 조회
query = """
SELECT * 
FROM movies
WHERE year >= 2010
"""

spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  4|겨울왕국| 디즈니|2014|    1| 16|
+---+--------+-------+----+-----+---+



In [23]:
# spark.sql(query) : Transformation  /  show() : Action

In [24]:
# 제목이 ~맨으로 끝나는 영화 정보 조회

query = """
SELECT *
FROM movies
WHERE name Like '%맨'
"""

spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [25]:
# 개봉월이 4~8월 사이인 영화 정보 조회
query = """
select *
from movies
where month between 4 and 8
"""

spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [26]:
# 회사 이름이 '마'로 시작하거나 '니'로 끝나는 영화중 2010년 이후로 개봉한 영화
query = """
select *
from movies
where (company like '마%' or company like '%니') and year >= 2010 
"""

spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  4|겨울왕국| 디즈니|2014|    1| 16|
+---+--------+-------+----+-----+---+



In [27]:
# 개봉연도 오름차순 정렬
query = """
select *
from movies
order by year
"""
spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  3|  배트맨|     DC|2008|    8|  6|
|  5|아이언맨|   마블|2008|    4| 30|
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  4|겨울왕국| 디즈니|2014|    1| 16|
+---+--------+-------+----+-----+---+



In [28]:
# 회사별 몇개의 영화가 있는지 조회
query = """
select company, count(*) as movie_count
from movies
group by company
"""

spark.sql(query).show()

+-------+-----------+
|company|movie_count|
+-------+-----------+
| 디즈니|          1|
|   마블|          2|
|     DC|          2|
+-------+-----------+



Join

In [29]:
attendances = [
    (1, 13934592., "KR"),
    (2, 2182227.,"KR"),
    (3, 4226242., "KR"),
    (4, 10303058., "KR"),
    (5, 4300365., "KR")
]

In [30]:
# 스키마 직접 지정하기
from pyspark.sql.types import StringType, FloatType, IntegerType # Data Type

# StructField : 각 컬럼을 만들기 위한 집합
# StructType : 컬럼 집합
from pyspark.sql.types import StructType, StructField # 구조

In [31]:
att_schema = StructType([
    StructField('id', IntegerType(), True), #Null허용여부 : True
    StructField('att', FloatType(), True),
    StructField('country', StringType(), True)
])

In [32]:
att_df = spark.createDataFrame(
    data = attendances,
    schema = att_schema
)

att_df.dtypes

[('id', 'int'), ('att', 'float'), ('country', 'string')]

In [33]:
att_df.createOrReplaceTempView('att')
# 테이블을 만들어 스파크 세션에 등록

In [34]:
att_df.show()

+---+-----------+-------+
| id|        att|country|
+---+-----------+-------+
|  1|1.3934592E7|     KR|
|  2|  2182227.0|     KR|
|  3|  4226242.0|     KR|
|  4|1.0303058E7|     KR|
|  5|  4300365.0|     KR|
+---+-----------+-------+



In [35]:
# 영화 아이디, 영화 이름, 회사, 관객수 조회
query = '''
select movies.id , movies.name, movies.company, att.att
from movies 
join att on movies.id = att.id
'''
spark.sql(query).show()

+---+--------+-------+-----------+
| id|    name|company|        att|
+---+--------+-------+-----------+
|  1|어벤져스|   마블|1.3934592E7|
|  2|  슈퍼맨|     DC|  2182227.0|
|  3|  배트맨|     DC|  4226242.0|
|  4|겨울왕국| 디즈니|1.0303058E7|
|  5|아이언맨|   마블|  4300365.0|
+---+--------+-------+-----------+



# 데이터프레임 API

In [36]:
# collect를 사용하면 RDD처럼 등장
df.select("*").collect()

[Row(id=1, name='어벤져스', company='마블', year=2012, month=4, day=26),
 Row(id=2, name='슈퍼맨', company='DC', year=2013, month=6, day=13),
 Row(id=3, name='배트맨', company='DC', year=2008, month=8, day=6),
 Row(id=4, name='겨울왕국', company='디즈니', year=2014, month=1, day=16),
 Row(id=5, name='아이언맨', company='마블', year=2008, month=4, day=30)]

In [37]:
df.select('name', 'company').collect()

[Row(name='어벤져스', company='마블'),
 Row(name='슈퍼맨', company='DC'),
 Row(name='배트맨', company='DC'),
 Row(name='겨울왕국', company='디즈니'),
 Row(name='아이언맨', company='마블')]

In [38]:
df.select(df.name,(df.year-2000).alias('year')).show()

+--------+----+
|    name|year|
+--------+----+
|어벤져스|  12|
|  슈퍼맨|  13|
|  배트맨|   8|
|겨울왕국|  14|
|아이언맨|   8|
+--------+----+



In [39]:
# agg : Aggregate의 약자. 그룹핑 후 데이터를 하나로 합쳐주는 역할
df.agg({'id':'count'}).collect()

[Row(count(id)=5)]

In [40]:
# query에서 사용가능한 함수들을 모아놓은 패키지가 존재
from pyspark.sql import functions as F

df.agg(F.min(df.year)).collect()

[Row(min(year)=2008)]

In [41]:
# groupBy
df.groupBy().avg().collect() # 컬럼명이 부여되지 않으면 각 컬럼별로 집계를 수행한다. 

[Row(avg(id)=3.0, avg(year)=2011.0, avg(month)=4.6, avg(day)=18.2)]

In [42]:
# 회사 별 월 별 영화 개수 정보
df.groupBy([df.company, df.month]).count().collect()

[Row(company='디즈니', month=1, count=1),
 Row(company='DC', month=8, count=1),
 Row(company='DC', month=6, count=1),
 Row(company='마블', month=4, count=2)]

In [43]:
# 회사별 개봉월 평균
df.groupBy('company').agg({"month":"mean"}).collect()

[Row(company='디즈니', avg(month)=1.0),
 Row(company='마블', avg(month)=4.0),
 Row(company='DC', avg(month)=7.0)]

In [44]:
# join
df.join(att_df, 'id').select(df.name, att_df.att).show()

+--------+-----------+
|    name|        att|
+--------+-----------+
|어벤져스|1.3934592E7|
|  슈퍼맨|  2182227.0|
|  배트맨|  4226242.0|
|겨울왕국|1.0303058E7|
|아이언맨|  4300365.0|
+--------+-----------+



In [45]:
# select, where, orederBy 사용
df.select('name','company','year').where("company == '마블'").orderBy('id').collect()

[Row(name='어벤져스', company='마블', year=2012),
 Row(name='아이언맨', company='마블', year=2008)]

In [46]:
spark.stop()
sc.stop()